In [1]:
import pandas as pd
import numpy as np
from astropy.cosmology import Planck13
from astropy.io import fits
from astropy.table import Table

import disperse

In [2]:
!ls

clusters.ipynb		       metrics.json
coma_DR12_filtered_RaDecZ.csv  MGS_DR7
_disperse_01		       OLD
_disperse_02		       __pycache__
_disperse_03		       README.md
_disperse_04		       SDSS
disperse.py		       sdss_opt_gr_filtered_RaDecZ.csv
DR5_cluster-catalog_v1.1.fits  UNet3D.py
dumps			       UNet.ipynb
metrics.ipynb


In [3]:
!ls SDSS

DR5_cluster-catalog_v1.1.fits  SDSS_DR16.csv
SDSS_DR12_MGS.dat	       SDSS_DR7.csv
SDSS_DR12_optical_groups.dat   sdss_opt_gr_filtered_RaDecZ.csv


In [4]:
sdss_dr16 = pd.read_csv('SDSS/SDSS_DR16.csv')

In [5]:
sdss_dr16 = sdss_dr16[sdss_dr16['class'] == 'GALAXY']

In [6]:
sdss_dr16 = sdss_dr16[['ra', 'dec', 'z']]
sdss_dr16.columns = ['RA', 'DEC', 'Z']
sdss_dr16

,RA,DEC,Z
0,146.71421,-1.041304,0.021275
1,146.71421,-1.041304,0.021222
2,146.91945,-0.990492,0.213925
3,146.74413,-0.652191,0.203783
6,146.85983,-0.808902,0.126554
...,...,...,...
5789190,146.13969,26.500868,0.903536
5789192,146.20911,26.557960,0.628224
5789194,146.09398,26.524402,0.845268
5789197,146.32252,26.549261,0.058894


In [7]:
# clusters
dat = Table.read('DR5_cluster-catalog_v1.1.fits', format='fits')
clusters = dat[['RADeg', 'decDeg', 'redshift', 'redshiftType', 'M500c']].to_pandas()
clusters = clusters[clusters['redshiftType'] == b'spec']
clusters = clusters[['RADeg', 'decDeg', 'redshift', 'M500c']]
clusters.columns = ['RA', 'DEC', 'Z', 'M500c']
clusters = clusters[clusters['M500c'] > 3.8]
clusters.reset_index(drop=True, inplace=True)
clusters

,RA,DEC,Z,M500c
0,0.802528,-6.089387,0.233369,6.456959
1,0.935321,10.024777,0.369317,4.251252
2,1.436675,-37.859517,0.503000,4.210469
3,2.045439,2.019734,0.365953,4.554281
4,2.932544,-15.385916,0.378000,6.010753
...,...,...,...,...
260,354.412551,0.272715,0.276900,5.477216
261,355.323184,-9.020276,0.251183,3.949305
262,355.300385,-51.327833,1.003000,3.976591
263,356.186180,-42.720662,0.596000,9.807067


In [8]:
# ra_int = (80, 280)
# dec_int = (-20, 80)
# z_int = (0.1, 0.2)

# sdss_dr16 = sdss_dr16[
#         (ra_int[0] < sdss_dr16['RA']) & (sdss_dr16['RA'] < ra_int[1]) & \
#         (dec_int[0] < sdss_dr16['DEC']) & (sdss_dr16['DEC'] < dec_int[1]) & \
#         (z_int[0] < sdss_dr16['Z']) & (sdss_dr16['Z'] < z_int[1])
#     ]
# sdss_dr16.reset_index(drop=True, inplace=True)

# clusters = clusters[
#         (ra_int[0] < clusters['RA']) & (clusters['RA'] < ra_int[1]) & \
#         (dec_int[0] < clusters['DEC']) & (clusters['DEC'] < dec_int[1]) & \
#         (z_int[0] < clusters['Z']) & (clusters['Z'] < z_int[1])
#     ]
# clusters.reset_index(drop=True, inplace=True)

In [9]:
sdss_dr16.reset_index(drop=True, inplace=True)
sdss_dr16

,RA,DEC,Z
0,146.71421,-1.041304,0.021275
1,146.71421,-1.041304,0.021222
2,146.91945,-0.990492,0.213925
3,146.74413,-0.652191,0.203783
4,146.85983,-0.808902,0.126554
...,...,...,...
3234558,146.13969,26.500868,0.903536
3234559,146.20911,26.557960,0.628224
3234560,146.09398,26.524402,0.845268
3234561,146.32252,26.549261,0.058894


In [10]:
clusters

,RA,DEC,Z,M500c
0,0.802528,-6.089387,0.233369,6.456959
1,0.935321,10.024777,0.369317,4.251252
2,1.436675,-37.859517,0.503000,4.210469
3,2.045439,2.019734,0.365953,4.554281
4,2.932544,-15.385916,0.378000,6.010753
...,...,...,...,...
260,354.412551,0.272715,0.276900,5.477216
261,355.323184,-9.020276,0.251183,3.949305
262,355.300385,-51.327833,1.003000,3.976591
263,356.186180,-42.720662,0.596000,9.807067


In [11]:
cosmo = Planck13
H0 = cosmo.H0.value
Om = cosmo.Om0
Ol = 0.69288
Ok = 0.0
print(f'H0 = {cosmo.H0.value}')
print(f'Om = {cosmo.Om0}')
print(f'Ol = {0.69288}')

H0 = 67.77
Om = 0.30712
Ol = 0.69288


In [12]:
DPS = disperse.Disperse3D(
    sdss_dr16, '_disperse_03/bin/',
    H0, Om, Ol, Ok,
    clusters=clusters
)

In [13]:
DPS.count_cart_coords()

In [14]:
DPS.galaxies

,RA,DEC,Z,CX,CY,CZ
0,146.71421,-1.041304,0.021275,-78.275260,51.389450,-1.701964
1,146.71421,-1.041304,0.021222,-78.080609,51.261657,-1.697732
2,146.91945,-0.990492,0.213925,-752.483608,490.173976,-15.526524
3,146.74413,-0.652191,0.203783,-717.294251,470.383801,-9.764327
4,146.85983,-0.808902,0.126554,-454.757239,296.907043,-7.667998
...,...,...,...,...,...,...
3234558,146.13969,26.500868,0.903536,-2341.547516,1571.101261,1405.947736
3234559,146.20911,26.557960,0.628224,-1755.177993,1174.585158,1055.644418
3234560,146.09398,26.524402,0.845268,-2223.648340,1494.568831,1337.246770
3234561,146.32252,26.549261,0.058894,-191.282881,127.461228,114.850702


In [15]:
SIGMA = 5.0
SMOOTH = 1
BOARD = 'smooth'
ANGLE = 30

In [16]:
# DPS.apply_disperse(SIGMA, SMOOTH, BOARD, ANGLE)

In [17]:
# DPS.save(f'dumps/{SIGMA}/')

In [18]:
sigmas = [3.8, 4.0, 4.2, 4.4, 4.6, 4.8, 5.0, 5.2, 5.4, 5.6, 5.8, 6.0]

In [19]:
for sigma in sigmas:
    SIGMA = sigma
    print('### ', SIGMA)
    DPS.apply_disperse(SIGMA, SMOOTH, BOARD, ANGLE)
    DPS.save(f'dumps/{SIGMA}/')

###  3.8
>>> delaunay_3D starts
>>> mse starts
>>> skelconv starts
>>> read_skl_ascii_RaDecZ starts
###  4.0
>>> delaunay_3D starts
>>> mse starts
>>> skelconv starts
>>> read_skl_ascii_RaDecZ starts
###  4.2
>>> delaunay_3D starts
>>> mse starts
>>> skelconv starts
>>> read_skl_ascii_RaDecZ starts
###  4.4
>>> delaunay_3D starts
>>> mse starts
>>> skelconv starts
>>> read_skl_ascii_RaDecZ starts
###  4.6
>>> delaunay_3D starts
>>> mse starts
>>> skelconv starts
>>> read_skl_ascii_RaDecZ starts
###  4.8
>>> delaunay_3D starts
>>> mse starts
>>> skelconv starts
>>> read_skl_ascii_RaDecZ starts
###  5.0
>>> delaunay_3D starts
>>> mse starts
>>> skelconv starts
>>> read_skl_ascii_RaDecZ starts
###  5.2
>>> delaunay_3D starts
>>> mse starts
>>> skelconv starts
>>> read_skl_ascii_RaDecZ starts
###  5.4
>>> delaunay_3D starts
>>> mse starts
>>> skelconv starts
>>> read_skl_ascii_RaDecZ starts
###  5.6
>>> delaunay_3D starts
>>> mse starts
>>> skelconv starts
>>> read_skl_ascii_RaDecZ starts


In [20]:
# DPS = disperse.Disperse3D.read('dumps/5.0')

In [21]:
# def get_points_LOS(cx, cy, cz, cx_int, cy_int, cz_int):
#     l = (cx**2 + cy**2 + cz**2) ** 0.5
#     vector = [3 * cx/l, 3 * cy/l, 3 * cz/l]
#     print(vector)
#     CX, CY, CZ = [], [], []
#     cur_p = vector.copy()
#     while (cx_int[0] < cur_p[0] < cx_int[1]) \
#         and (cy_int[0] < cur_p[1] < cy_int[1]) \
#         and (cz_int[0] < cur_p[2] < cz_int[1]):
#         CX.append(cur_p[0])
#         CY.append(cur_p[1])
#         CZ.append(cur_p[2])
#         cur_p[0] += vector[0]
#         cur_p[1] += vector[1]
#         cur_p[2] += vector[2]
#     df = pd.DataFrame()
#     df = df.assign(CX=CX)
#     df = df.assign(CY=CY)
#     df = df.assign(CZ=CZ)
    
#     return df

In [22]:
# points = get_points_LOS(
#     DPS.clusters.iloc[0]['CX'],
#     DPS.clusters.iloc[0]['CY'],
#     DPS.clusters.iloc[0]['CZ'],
#     DPS.CX_int, DPS.CY_int, DPS.CZ_int
# )

In [23]:
# len(points)

In [24]:
# c = DPS.count_conn([3] * points.shape[0], points)